In [1]:
import json
from azure.eventhub import EventHubProducerClient, EventData
from dotenv import load_dotenv
import os
import socket
import random

from random import randrange

load_dotenv()
EVENT_HUB_CONN_STRING = os.getenv("EVENT_HUB_CONN_STRING") 
EVENT_HUB_NAME_IMPRESSIONS = os.getenv("EVENT_HUB_NAME_IMPRESSIONS")
EVENT_HUB_NAME_CLICKS = os.getenv("EVENT_HUB_NAME_CLICKS")

eventHubConnString = EVENT_HUB_CONN_STRING
eventHubNameImpressions = EVENT_HUB_NAME_IMPRESSIONS
eventHubNameClicks = EVENT_HUB_NAME_CLICKS

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

hostname = socket.gethostname()

In [2]:
from utils import generateClickEvent, generateImpressionEvent

def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

def generateEvents(isUnsupportedBrowser = False, isAnomaly = False):
    try:
        while True:
            impressionEvent = generateImpressionEvent(isUnsupportedBrowser, isAnomaly, None)    
            sendToEventsHub(impressionEvent, producer_impressions)
            if random.randint(1, 100) > 80:
                clickEvent = generateClickEvent(impressionEvent)    
                sendToEventsHub(clickEvent, producer_clicks)
    except KeyboardInterrupt:
        producer_impressions.close()
        producer_clicks.close()


In [3]:
generateEvents(False, False)

{"impressionId": "b82c74d1-1f49-4b11-a515-4276cfb4f500", "impressionDate": "2024-01-06T03:27:10", "productId": 797, "browser": "Firefox", "browserVersion": "13.6", "device": "computer", "source": "bing", "ip_address": "208.15.61.99", "landing_page": "http://clark.com/blog/posts/mainhome.html", "page_loading_seconds": 5.37}
{"clickId": "2d149d3f-dff5-497d-ac07-61407d375656", "impressionId": "b82c74d1-1f49-4b11-a515-4276cfb4f500", "clickDate": "2024-01-06T03:27:10", "productId": 797}
{"impressionId": "70686220-f22b-4222-920f-dfad1295e7cb", "impressionDate": "2024-01-03T12:10:21", "productId": 792, "browser": "Chrome", "browserVersion": "6.6", "device": "computer", "source": "bing", "ip_address": "180.131.118.240", "landing_page": "https://www.hancock-bailey.info/explore/searchlogin.htm", "page_loading_seconds": 4.693}
{"impressionId": "22b1e52b-a8a7-497d-8429-fbda341b7344", "impressionDate": "2024-01-02T06:11:32", "productId": 960, "browser": "Edge", "browserVersion": "4.5", "device": "m